In [1]:
# Imported important files

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [31]:
# Read the Excel and displayed the first five rows

df = pd.read_excel('EBERECHI MAUREEN NWANKWO_8145497791_20250416095037.xlsx')
df.head()

,Trans. Time,Value Date,Description,Debit/Credit(₦),Balance(₦),Channel,Transaction Reference
0,2024 Mar 21 22:12:\n19,2024-03-21,Add Money-Bank USSD,13300.0,13300.0,E-Channel,240321031864561528
1,2024 Mar 25 12:48:\n24,2024-03-25,Transfer to Shehu umar provisions store,-3500.0,9800.0,E-Channel,240325026876769097
2,2024 Mar 25 13:18:\n26,2024-03-25,Transfer from NWANKWO EBERECHI MAUREEN,3500.0,13300.0,E-Channel,000016240325141824000010482401
3,2024 Mar 27 08:\n59:49,2024-03-27,Transfer to DE-CREATIVE ARTS CONSULT AND PRI,-4500.0,8800.0,E-Channel,240327020437473355
4,2024 Mar 28 08:\n37:58,2024-03-28,Transfer to FORBBIES HAIRS,-5000.0,3800.0,E-Channel,240328027010858986


In [3]:
# Statistical Description of the numerical columns

df.describe()

,Debit/Credit(₦),Balance(₦)
count,268.000000,268.000000
mean,18.733209,12307.449627
std,4586.656917,10136.444504
min,-21000.000000,0.000000
25%,-2000.000000,5087.500000
50%,500.000000,10343.750000
75%,1400.000000,16211.500000
max,30000.000000,47299.000000


In [4]:
# Number of rows and columns

df.shape

(498, 7)

# Data Preparation

In [5]:
# number of null values in each of the columns

df1 = df.isnull().sum()
df1

Trans. Time                3
Value Date               230
Description              227
Debit/Credit(₦)          230
Balance(₦)               230
Channel                  230
Transaction Reference    223
dtype: int64

In [33]:
# Total Credit/Debit Values respectively

# Calculate total credit (positive values)
total_credit = df[df['Debit/Credit(₦)'] > 0]['Debit/Credit(₦)'].sum()

# Calculate total debit (negative values, we'll make it positive for reporting)
total_debit = abs(df[df['Debit/Credit(₦)'] < 0]['Debit/Credit(₦)'].sum())

print(f"Total Credit: ₦{total_credit:.2f}")
print(f"Total Debit: ₦{total_debit:.2f}")

Total Credit: ₦380050.00
Total Debit: ₦375029.50


In [34]:
# Spliting the credit and debit in different columns

df['total_credit'] = df[df['Debit/Credit(₦)'] > 0]['Debit/Credit(₦)']
df['total_debit'] = abs(df[df['Debit/Credit(₦)'] < 0]['Debit/Credit(₦)'])

In [35]:
# filled up the null values of the credit and debit column

df['total_credit']= df['total_credit'].ffill()
df['total_debit']= df['total_debit'].ffill()

In [36]:
# top five credit sources

top_5_credit = df.nlargest(5,'total_credit')
top_5_credit

,Trans. Time,Value Date,Description,Debit/Credit(₦),Balance(₦),Channel,Transaction Reference,total_credit,total_debit
234,2024 Jul 23 21:00:,2024-07-23,Transfer from DANIEL CHIKE EZEALA,30000.0,30999.0,E-Channel,000004240723220052215989635019,30000.0,1500.0
235,27,NaT,NaN,NaN,NaN,NaN,NaN,30000.0,1500.0
236,2024 Jul 24 10:15:31,2024-07-24,Transfer to JOHN EJELEKWU STEVEN,-2800.0,28199.0,E-Channel,240724015071038403,30000.0,2800.0
237,2024 Jul 24 18:40:,2024-07-24,Transfer to CHIMA STEPHEN OKEKE,-3500.0,24699.0,E-Channel,100004240724184042117153219660,30000.0,3500.0
238,34,NaT,NaN,NaN,NaN,NaN,NaN,30000.0,3500.0


In [37]:
top_5_debit = df.nlargest(5,'total_debit')
top_5_debit

,Trans. Time,Value Date,Description,Debit/Credit(₦),Balance(₦),Channel,Transaction Reference,total_credit,total_debit
363,2024 Oct 08 15:20:,2024-10-08,"Transfer to STEVEN, EJELEKWU JOHN",-21000.0,2249.0,E-Channel,100004241008152059120364981656,7000.0,21000.0
364,49,NaT,NaN,NaN,NaN,NaN,NaN,7000.0,21000.0
365,2024 Oct 08 18:21:,2024-10-08,Transfer from CHIMA STEPHEN OKEKE,3000.0,5249.0,E-Channel,000004241008192123913347694239,3000.0,21000.0
366,23,NaT,NaN,NaN,NaN,NaN,NaN,3000.0,21000.0
367,2024 Oct 08 18:30:,2024-10-08,Transfer from DANIEL CHIKE EZEALA,5000.0,10249.0,E-Channel,000014241008193022254789107996,5000.0,21000.0


In [29]:
# filling the null values in channel and balance columns

df4 = df['Balance(₦)'].median()
df['Balance(₦)'] = df['Balance(₦)'].fillna(df4)

df = df[['Value Date', 'Channel' ]].ffill()

In [10]:
# converting the date column to datetime

df['Value Date'] = pd.to_datetime(df['Value Date'])

# Data Exploration

In [11]:
# Exploring the rate of income per month

df['Value Month'] = df['Value Date'].dt.month

# aggregate total credit by value month
monthly_value = df.groupby('Value Month')['total_credit'].sum()

fig = go.Figure(data=[
    go.Scatter(
        x=monthly_value.index,
        y=monthly_value.values,
        marker=dict(color='red'),
        line=dict(color='blue')
    )
])

fig.update_layout(
    title='Total Monthly Income',
    xaxis_title='Month',
    yaxis_title='Total Credit',
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    ),
    height=600,
    width=1000
)

fig.show()

KeyError: 'Column not found: total_credit'

In [ ]:
# exploring the rate of expenses by month 

# aggregate total debit by Value month
monthly_value = df.groupby('Value Month')['total_debit'].sum()

fig = go.Figure(data=[
    go.Scatter(
        x=monthly_value.index,
        y=monthly_value.values,
        marker=dict(color='red'),
        line=dict(color='blue')
    )
])

fig.update_layout(
    title='Total Monthly Debit',
    xaxis_title='Month',
    yaxis_title='Total Debit',
    xaxis=dict(
        tickmode='array',
        tickvals=list(range(1, 13)),
        ticktext=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    ),
    height=600,
    width=1000
)

fig.show()

In [ ]:
# exploring the total credit per weekday

df['Value Day'] = df['Value Date'].dt.day_name()

weekday_value = df['Value Day'].value_counts().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

# aggregate total credit by day of the week
weekday_inflow = df.groupby('Value Day')['total_credit'].sum().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=weekday_value.index,
        y=weekday_value.values,
        name='Income',
        marker_color='blue',
        opacity=0.6,
        yaxis='y1'
    )
)

fig.add_trace(
    go.Scatter(
        x=weekday_inflow.index,
        y=weekday_inflow.values,
        name='Total Credit',
        mode='lines+markers',
        marker=dict(color='red'),
        line=dict(color='red'),
        yaxis='y2'
    )
)

fig.update_layout(
    title='Weekly Cash Inflow',
    xaxis=dict(
        title='Day of the Week',
        categoryorder ='array',
        categoryarray=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    ),
    yaxis=dict(
        title='Total Income',
        showgrid=False,
        side='left',
    ),
    yaxis2=dict(
        title='Total Income',
        overlaying='y',
        side='right',
        showgrid=False
    ), 
    legend=dict(
        x=1.05,
        y=1,
        orientation='v',
        xanchor='left'
    ),
    height=600,
    width=1000
)


fig.show()

In [ ]:
# exploring the total credit per weekday


weekday_inflow = df.groupby('Value Day')['total_debit'].sum().reindex(
    ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
)

fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=weekday_value.index,
        y=weekday_value.values,
        name='Expense',
        marker_color='red',
        opacity=0.6,
        yaxis='y1'
    )
)

fig.add_trace(
    go.Scatter(
        x=weekday_inflow.index,
        y=weekday_inflow.values,
        name='Total Debit',
        mode='lines+markers',
        marker=dict(color='blue'),
        line=dict(color='blue'),
        yaxis='y2'
    )
)

fig.update_layout(
    title='Weekly Cash Outflow',
    xaxis=dict(
        title='Day of the Week',
        categoryorder ='array',
        categoryarray=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    ),
    yaxis=dict(
        title='Total Expenses',
        showgrid=False,
        side='left',
    ),
    yaxis2=dict(
        title='Total Expenses',
        overlaying='y',
        side='right',
        showgrid=False
    ), 
    legend=dict(
        x=1.05,
        y=1,
        orientation='v',
        xanchor='left'
    ),
    height=600,
    width=1000
)


fig.show()